In [11]:
# 1. Read the ref-dist-mat into numpy for array
# 2. Replace array names with annotations
# 3. Double for loop to calculate F1 and F2
# 4. Use F measuremnts to calculated M and FST measures

In [12]:
import numpy as np

In [13]:
#dist_mat = np.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/data/Clupea_11jan24/11jan_jc_ref-dist-mat.txt.txt", dtype="str")
dist_mat = np.genfromtxt("/Users/sjr729/Desktop/GitHub/Gskimming/00_data/Skmer/Oedulis/RawCov/MDS/distance_matrix.txt", dtype="str")

#annots = np.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/data/Clupea_11jan24/11jan_ClupeaAtmore_Fstannot.tsv", dtype="str")
annots = np.genfromtxt("/Users/sjr729/Desktop/GitHub/Gskimming/01_infofiles/oedulis/oedulis_skmer1_Fstannot.tsv", dtype="str")

# dist_mat = numpy.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/fst_test_data/test_matrix.txt", dtype="str")
# annots = numpy.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/fst_test_data/test_annots.txt", dtype="str")

In [14]:
for line in annots:
    x = np.where(dist_mat == line[0])
    for index in x: 
        try:
            dist_mat[index[0]][index[1]] = line[1]
        except: pass

In [15]:
dist_dict = dict()

for num1 in range(1, len(dist_mat[0])):
    for num2 in range(1, len(dist_mat[0])):
        if num1 != num2: 
            name1=dist_mat[0][num1]
            name2=dist_mat[0][num2]
            if name1 not in dist_dict:
                dist_dict[name1] = dict()

            if name2 not in dist_dict[name1]:
                dist_dict[name1][name2] = [float(dist_mat[num1, num2])]
            else: 
                dist_dict[name1][name2].append(float(dist_mat[num1, num2]))
        if num1 == num2:
            name1=name2=dist_mat[0][num1]
            
            if name1 not in dist_dict:
                dist_dict[name1] = dict()

            if name2 not in dist_dict[name1]:
                dist_dict[name1][name2] = []

key_list = list(dist_dict.keys())
print(key_list)

['HALS', 'HAFR', 'AGAB', 'DOLV', 'CRES', 'WADD', 'VENO', 'BUNN', 'TOLL', 'THIS', 'RYAN', 'OSTR', 'BOVA', 'ORNE', 'NELL', 'MOLU', 'LANG', 'KLEV', 'HFJO', 'BARR', 'GREV', 'CORS', 'PONT', 'NISS', 'LOGS', 'LILL', 'INNE', 'HAUG', 'GASO', 'FURI', 'ZECE', 'CLEW', 'TRAL']


In [16]:
def make_empty_matrix(dist_dict, key_list):
    matrix = [[0 for x in range(0, len(dist_dict)+1)] for x in range(0, len(dist_dict)+1)]
    matrix[0][0]="sample"
    for x in range(0, len(dist_dict)):
        matrix[0][x+1] = matrix[x+1][0] = (key_list[x])
    return matrix

def calculate_WC_FST(F1, F2, M):
    numerator_wc_fst = F1 + F2
    denominator_wc_fst = F1 + F2 + 2 * (1 / (M + 1)) * (M * (1 - F1) + (1 - F2))
    WC_FST = numerator_wc_fst / denominator_wc_fst
    return WC_FST

def calculate_Nei_FST(F1, F2):
    numerator_nei_fst = F1 + F2
    denominator_nei_fst = 2 - ((F1 + F2) / 2)
    Nei_FST = numerator_nei_fst / denominator_nei_fst
    return Nei_FST

def calculate_Hudson_FST(F1, F2):
    Hudson_FST = (F1 + F2) / 2
    return Hudson_FST

In [17]:
F_mtrx = make_empty_matrix(dist_dict, key_list)
wh_fst_mtrx = make_empty_matrix(dist_dict, key_list)
wc_fst_mtrx = make_empty_matrix(dist_dict, key_list)
nei_fst_mtrx = make_empty_matrix(dist_dict, key_list)
hud_fst_mtrx = make_empty_matrix(dist_dict, key_list)
clade_dist_mtrx = make_empty_matrix(dist_dict, key_list)
thetas_list = [[key,0.0] for key in key_list]
print(thetas_list[1][1])

0.0


In [18]:
for key1, index1 in zip(key_list, range(len(key_list))):
     for key2, index2 in zip(key_list, range(len(key_list))):
        if key1 == key2:
            break
        else:
            dist_array_k1_k1 = dist_dict[key1][key1]
            dist_array_k2_k2 = dist_dict[key2][key2]
            dist_array_k1_k2 = dist_dict[key1][key2]

            across_k1_k2 = sum(dist_array_k1_k2)/len(dist_array_k1_k2)

            max_k1_k1 = max(dist_array_k1_k1) if (len(dist_array_k1_k1) != 0) else 0
            max_k2_k2 = max(dist_array_k1_k1) if (len(dist_array_k1_k1) != 0) else 0
            max_k1_k2 = max(dist_array_k1_k2) 

            clade_dist_mtrx[index1+1][index2+1] = clade_dist_mtrx[index2+1][index1+1] = max_k1_k2 - (max_k1_k1 + max_k2_k2)/2
 
            if ((len(dist_array_k1_k1) == 0) or len(dist_array_k2_k2) == 0):
                # Checks if any of the populations only have one members.
                # Calculates F1 or F2 if possible. If not, assigns "NA". 
                # Assigns "NA" to all other Fst calculations.
                if (len(dist_array_k1_k1) != 0):
                    within_k1 = sum(dist_array_k1_k1)/len(dist_array_k1_k1)
                    F1 = 1 - within_k1/across_k1_k2
                    thetas_list[index1][1] = round(within_k1, 5)
                    F_mtrx[index1+1][index2+1] = round(F1, 5)
                    F_mtrx[index2+1][index1+1] = "NA"
                elif (len(dist_array_k2_k2) != 0):
                    within_k2 = sum(dist_array_k2_k2)/len(dist_array_k2_k2)
                    F2 = 1- within_k2/across_k1_k2
                    thetas_list[index1][1] = "NA"
                    F_mtrx[index1+1][index2+1] = "NA"
                    F_mtrx[index2+1][index1+1] = round(F2, 5)
                    
                else:
                    F_mtrx[index1+1][index2+1] = F_mtrx[index2+1][index1+1] ="NA"

                wh_fst_mtrx[index1+1][index2+1] = wh_fst_mtrx[index2+1][index1+1] = "NA"
                wc_fst_mtrx[index1+1][index2+1] = wc_fst_mtrx[index2+1][index1+1] = "NA"
                nei_fst_mtrx[index1+1][index2+1] = nei_fst_mtrx[index2+1][index1+1] = "NA"
                hud_fst_mtrx[index1+1][index2+1] = hud_fst_mtrx[index2+1][index1+1] = "NA"
            else:
                within_k1 = sum(dist_array_k1_k1)/len(dist_array_k1_k1)
                within_k2 = sum(dist_array_k2_k2)/len(dist_array_k2_k2)

                F1 = 1 - within_k1/across_k1_k2
                F2 = 1 - within_k2/across_k1_k2

                if len(dist_array_k1_k1) <= len(dist_array_k2_k2):
                    M=len(dist_array_k1_k1)/len(dist_array_k2_k2)
                else:
                    M=len(dist_array_k2_k2)/len(dist_array_k1_k1)
                
                thetas_list[index1][1] = round(within_k1, 5)
                # NOTE: F_matrix contains F1 in the lower half and F2 in the upper half. 
                F_mtrx[index1+1][index2+1] =  round(F1, 5)
                F_mtrx[index2+1][index1+1] =  round(F2, 5)
                wh_fst_mtrx[index1+1][index2+1] = wh_fst_mtrx[index2+1][index1+1] = round((F1 + F2)/2, 5)
                wc_fst_mtrx[index1+1][index2+1] = wc_fst_mtrx[index2+1][index1+1] = round(calculate_WC_FST(F1, F2, M), 5)
                nei_fst_mtrx[index1+1][index2+1] = nei_fst_mtrx[index2+1][index1+1] = round(calculate_Nei_FST(F1, F2), 5)
                hud_fst_mtrx[index1+1][index2+1] = hud_fst_mtrx[index2+1][index1+1] =  round(calculate_Hudson_FST(F1, F2), 5)
for x in wh_fst_mtrx:
    print(x)

['sample', 'HALS', 'HAFR', 'AGAB', 'DOLV', 'CRES', 'WADD', 'VENO', 'BUNN', 'TOLL', 'THIS', 'RYAN', 'OSTR', 'BOVA', 'ORNE', 'NELL', 'MOLU', 'LANG', 'KLEV', 'HFJO', 'BARR', 'GREV', 'CORS', 'PONT', 'NISS', 'LOGS', 'LILL', 'INNE', 'HAUG', 'GASO', 'FURI', 'ZECE', 'CLEW', 'TRAL']
['HALS', 0, 0.06619, 0.1281, 0.11012, 0.20762, 0.19024, 0.00191, 0.02551, 0.07008, 0.01298, 0.11084, 0.27201, 0.02427, 0.01949, 0.35374, 0.25774, 0.0324, 0.014, 0.02978, 0.07274, 0.08998, 0.24131, 0.13603, 0.03922, -0.00369, 0.03183, 0.1076, 0.00718, 0.06609, 0.0062, 0.19555, 'NA', 0.14177]
['HAFR', 0.06619, 0, 0.11261, 0.0163, 0.16868, 0.19373, 0.04519, 0.02764, 0.07529, 0.0701, 0.04049, 0.25669, 0.02596, 0.02179, 0.34336, 0.22528, 0.02649, 0.05608, 0.06988, 0.1078, 0.02875, 0.12319, 0.15639, 0.00673, 0.03375, 0.08907, 0.07551, 0.02205, 0.11055, 0.07282, 0.1832, 'NA', 0.06116]
['AGAB', 0.1281, 0.11261, 0, 0.11497, 0.20783, 0.257, 0.10487, 0.10631, 0.14871, 0.11153, 0.15807, 0.12872, 0.09169, 0.09609, 0.38217, 0.260

In [19]:
np.savetxt("/Users/sjr729/Desktop/GitHub/Gskimming/00_data/Skmer/Oedulis/RawCov/Theta/oedulis_skmer1_n214_thetas.txt", np.matrix(thetas_list), fmt="%s", delimiter="\t")
#np.savetxt("./ed_test_Fmat.txt", np.matrix(F_mtrx), fmt="%s", delimiter="\t")
#np.savetxt("./ed_test_whfst.txt", np.matrix(wh_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("/Users/sjr729/Desktop/GitHub/Gskimming/00_data/Skmer/Oedulis/RawCov/Fst/oedulis_skmer1_n214_wcfst.txt", np.matrix(wc_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("/Users/sjr729/Desktop/GitHub/Gskimming/00_data/Skmer/Oedulis/RawCov/Fst/oedulis_skmer1_n214_neifst.txt", np.matrix(nei_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("/Users/sjr729/Desktop/GitHub/Gskimming/00_data/Skmer/Oedulis/RawCov/Fst/oedulis_skmer1_n214_hudfst.txt", np.matrix(hud_fst_mtrx), fmt="%s", delimiter="\t")
#np.savetxt("./ed_test_cladedist.txt", np.matrix(clade_dist_mtrx), fmt="%s", delimiter="\t")